In [ ]:
# Setup
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu
!pip install -q langchain langchain-community
!pip install -qU pypdf
!pip install gradio

import torch
import locale
from google.colab import drive

locale.getpreferredencoding = lambda: "UTF-8"

drive.mount('/content/drive/')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 

In [ ]:
# Prepare relevant files
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFDirectoryLoader('/content/drive/MyDrive/My Projects/Files/Course Selection Assistant (Prototype)')
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=51)
"""
worse performance:
  chunk_size=512, chunk_overlap=30
  chunk_size=256, chunk_overlap=25
"""

files = []
async for file in loader.alazy_load():
    files.append(file)
print(f"{files[-1].metadata}\n")
print(files[-1].page_content)

chunked_docs = splitter.split_documents(files)

{'source': '/content/drive/MyDrive/My Projects/Files/Course Selection Assistant (Prototype)/UNDERGRADUATE PROGRAMS.pdf', 'page': 18}

also be nurtured through the study of arts, humanities 
and cultural studies. The program aims to develop 
students as intellectuals. They learn to appreciate and 
respect cultural diversities, follow their heart and 
aspirations in their endeavors, serve our communities, 
and take the lead to advance social justice and the 
well-being of mankind.
CC Website



In [ ]:
# Initialize vector database & retriever
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

vector_db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))
"""
not working:
  model_name="dunzhang/stella_en_400M_v5"
  model_name="Marqo/dunzhang-stella_en_400M_v5"
  model_name="Alibaba-NLP/gte-large-en-v1.5"
worse performance:
  model_name="Snowflake/snowflake-arctic-embed-l"
"""
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})
"""
worse performance:
  "k": 4
  "k": 3
"""

<ipython-input-3-edc9e1019299>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vector_db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face H

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

'\nworse performance:\n  "k": 4\n  "k": 3\n'

In [ ]:
# Initialize model & tokenizer
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

model_name = "tanliboy/lambda-qwen2.5-14b-dpo-test"
"""
not working:
  "upstage/solar-pro-preview-instruct"
worse performance:
  "HuggingFaceH4/zephyr-7b-beta"
"""

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/709 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/47.5k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

In [ ]:
# Initialize LLM
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

<ipython-input-5-2aa79e53ddf0>:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [ ]:
# Build RAG chain
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt_template = """
<|system|>

You're a friendly career consultant at an university who answers students' question s related to course selection. Answer the question with course names in \
bullet points based on your knowledge and the information provided by the user, if any. Surround important keywords with the "*" symbol. Include the names \
of the relevant sections. Do not include any links. Include relevant jobs that might require the skills taught in the courses. End by asking the user \
follow-up questions related to their personality, preferences, or goals. Use the following context to help:
{context}
</s>

<|user|>

{question}
</s>

<|assistant|>

"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

In [ ]:
# Answer questions based on given context
question1 = "What does auditing a course mean?"
response1 = rag_chain.invoke(question1)

print(response1.split("</s>")[2].strip()[15:])

Auditing a course means attending classes and participating in activities, such as writing the final examination, without receiving a grade or earning credits for the course. Here's how it works:

- You need "*permission*" from the course instructor.
- Activities and participation should be "*mutually agreed*" upon during class enrollment.
- Some schools or departments might "*exclude*" auditors from certain courses or impose extra restrictions.

If you successfully meet the requirements set by the instructor, the course will appear as "AU" on your transcript. However, if you don't fulfill these requirements, the course won’t be recorded on your transcript.

This option could be useful if you want to learn about a topic without needing formal credit, or if you're considering taking the course again in future semesters to earn credits.

Would you like to know more about specific courses that allow auditing? Or perhaps you'd prefer details on prerequisites and co-requisites for certain c

In [ ]:
question2 = "What are the foundation course requirements for the School of Science?"
response2 = rag_chain.invoke(question2)

print(response2.split("</s>")[2].strip()[15:])

The ***foundation course*** requirements for the School of Science include:

- **Computer Course**: 
    - *COMP 1021 Introduction to Computer Science*
    - *COMP 1022P Introduction to Computing with Java*
    - *COMP 2011 Programming with C++*

- **Science Foundation Courses**:
    - You need to take 8 courses from the specified elective list.
    - Take 7 ***foundation lecture courses***, ensuring you have at least 1 but no more than 3 lecture courses from each of these four disciplines: ***CHEM***, ***LIFS/OCES***, ***MATH***, and ***PHYS***.
    - Additionally, complete 1 ***laboratory course***.

These courses equip you with foundational skills necessary for majors like ***Physics***, ***Chemistry***, ***Data Analytics in Science***, ***Ocean Science and Technology***, and interdisciplinary programs such as ***Data Science and Technology***, ***Biotechnology and Business***, ***Mathematics and Economics***, and ***Risk Management and Business Intelligence***.

Would you like to e

In [ ]:
question3 = "What are the major requirements for the School of Engineering?"
response3 = rag_chain.invoke(question3)

print(response3.split("</s>")[2].strip()[15:])

To fulfill the major requirements for the ***School of Engineering*** (***SENG***) at HKUST, you need to consider both the general graduation requirements and specific major prerequisites. Here's a breakdown:

- **General Graduation Requirements**: These include fulfilling the credit requirements as specified in the [Graduation Requirements](https://prog-crs.hkust.edu.hk/ugprog). This typically involves completing a certain number of credits across various categories.

- **Major Prerequisites and Specific Courses**:
    - For most programs in ***SENG*** excluding ***BIEN*** (Bioengineering), ***CEEV*** (Civil Engineering), ***CENG*** (Chemical Engineering), and ***COSC*** (Computer Science and Engineering) programs, you should focus on courses under the ***Technology (T)*** category.
    - If you are enrolled in one of these excluded programs (***BIEN***, ***CEEV***, ***CENG***, or ***COSC***), you will have to take courses from both the ***Science (S)*** and ***Technology (T)*** categ

In [ ]:
question4 = "What are the major requirements for the School of Business and Management?"
response4 = rag_chain.invoke(question4)

print(response4.split("</s>")[2].strip()[15:])

The ***major*** requirements for the School of Business and Management typically include foundational courses that provide a broad understanding of business principles. Based on the available information:

- **Operations Management** (ISOM 2700) - This course covers how businesses manage processes to create goods and services efficiently.
  
- **Marketing Management** (MARK 2120) - Here you'll learn about marketing strategies and consumer behavior.

- **Organizational Behavior** (MGMT 2110) - Focuses on understanding individual and group dynamics within organizations.

These courses equip students with essential skills needed for various roles such as *operations manager*, *marketing analyst*, or *human resources specialist*.

Could you tell me more about your interests or career aspirations so I can recommend additional courses?


In [ ]:
# Abstraction
import gradio as gr

def invoke_rag(question):
    # if not question: raise Exception("Question field empty")
    if not question: return "Feel free to ask me any questions regarding course selection."

    response = rag_chain.invoke(question)
    return response.split("</s>")[2].strip()[15:]

callback1 = gr.CSVLogger()
callback2 = gr.CSVLogger()
callback3 = gr.CSVLogger()

def receive_feedback1(question, response):
    callback1.flag([question, response])
    return "Thank you for your feedback!"

def receive_feedback2(question, response):
    callback2.flag([question, response])
    return "Thank you for your feedback!"

def receive_feedback3(question, response):
    callback3.flag([question, response])
    return "Thank you for your feedback!"

In [ ]:
# Build demo

with gr.Blocks() as demo:
    gr.HTML("<center><h1>HKUST IntelliChat</h1></center>")

    question = gr.Textbox(label="Question", placeholder="Please enter your question here...", lines=1)
    response = gr.Textbox(label="Assistant Response", lines=25)
    submit = gr.Button("Get answer")
    with gr.Row():
        feedback1 = gr.Button("Answer was useful")
        feedback2 = gr.Button("Answer was inaccurate")
        feedback3 = gr.Button("Answer was irrelevant")

    callback1.setup([question, response], "useful")
    callback2.setup([question, response], "inaccurate")
    callback3.setup([question, response], "irrelevant")

    gr.on(
        triggers=[question.submit, submit.click],
        fn=invoke_rag,
        inputs=question,
        outputs=response
    )
    feedback1.click(receive_feedback1, [question, response], [response], preprocess=False)
    feedback2.click(receive_feedback2, [question, response], [response], preprocess=False)
    feedback3.click(receive_feedback3, [question, response], [response], preprocess=False)

# demo.launch(share=True, debug=True)
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cdf5fb63d0bcea344f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
demo.close()

Closing server running on port: 7860
